# Impact-T fieldmap reconstruction

In [ ]:
from impact import Impact, fieldmaps
from pmd_beamphysics.interfaces.impact import create_fourier_coefficients
import numpy as np

In [ ]:
ifile = "templates/lcls_injector/ImpactT.in"
I = Impact(input_file=ifile)

In [ ]:
# Fieldmaps are stored here
I.input["fieldmaps"].keys()

In [ ]:
# Look at a solrf element.
I.ele["SOL1"]

In [ ]:
# This is its fieldmap filename
I.ele["SOL1"]["filename"]

In [ ]:
# That data is here.
fmap = I.input["fieldmaps"]["rfdata102"]
fmap

In [ ]:
# Reconstruction function
fieldmaps.fieldmap_reconstruction_solrf(fmap["field"]["Bz"], 0)

# Basic plot

In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
zlist = np.linspace(0, 0.49308, 1000)
fieldlist = [
    fieldmaps.fieldmap_reconstruction_solrf(fmap["field"]["Bz"], z) for z in zlist
]

In [ ]:
# z at max field
zlist[np.argmax(np.array(fieldlist))]

In [ ]:
plt.plot(zlist, fieldlist);

In [ ]:
# Integrated field (approximate)
field_scale = 0.243  # from imput file
BL = np.sum(fieldlist) * 0.49308 / 1000  # T*m
BL * field_scale * 10  # T*m -> kG*m

In [ ]:
1 / BL

# Create Fieldmap

In [ ]:
fmap2 = fmap.copy()
fmap2["field"]["Bz"]["z0"] = min(zlist)
fmap2["field"]["Bz"]["z1"] = max(zlist)
fmap2["field"]["Bz"]["L"] = np.ptp(zlist)
fmap2["field"]["Bz"]["fourier_coefficients"] = create_fourier_coefficients(
    zlist, fieldlist, n=20
)

In [ ]:
fieldlist2 = [
    fieldmaps.fieldmap_reconstruction_solrf(fmap2["field"]["Bz"], z) for z in zlist
]
plt.plot(zlist, fieldlist, label="original")
plt.plot(zlist, fieldlist2, "--", label="created")
plt.legend()

In [ ]:
fmap2

In [ ]:
from numpy import sin, cos, pi, arange

In [ ]:
# Raw data from
coefs = fmap2["field"]["Bz"]["fourier_coefficients"]
coefs

In [ ]:
A0 = coefs[0]  # constant factor
A = coefs[1::2]  # cos parts
B = coefs[2::2]  # sin parts

In [ ]:
L = 0.6


@np.vectorize
def f(z):
    phase = 2 * pi * (z / L - 1 / 2)
    return A0 / 2 + sum(
        [
            A[n - 1] * cos(n * phase) + B[n - 1] * sin(n * phase)
            for n in arange(1, len(A) + 1)
        ]
    )


z0 = np.linspace(0, L, 100)

plt.plot(z0, f(z0))